# 3. Feature engineering & préparation des données pour la modélisation

Objectifs :
1. transformer les colonnes brutes (oui/non, pourcentages…)
2. créer des variables dérivées pertinentes (features métier)
3. contrôler les incohérences potentielles
4. produire un dataset final prêt pour scikit-learn (X/y ensuite)


In [1]:
import pandas as pd

from technova_attrition.config import PATHS
from technova_attrition.features import add_engineered_features, compute_incoherence_metrics
from technova_attrition.stats_tests import compare_groups_univariate

df = pd.read_parquet(PATHS.data_processed / "employees_joined.parquet")
df.shape, df.head()

((1470, 35),
    age genre  revenu_mensuel statut_marital departement  \
 0   41     F            5993    Célibataire  Commercial   
 1   49     M            5130       Marié(e)  Consulting   
 2   37     M            2090    Célibataire  Consulting   
 3   33     F            2909       Marié(e)  Consulting   
 4   27     M            3468       Marié(e)  Consulting   
 
                     poste  nombre_experiences_precedentes  \
 0        Cadre Commercial                               8   
 1  Assistant de Direction                               1   
 2              Consultant                               6   
 3  Assistant de Direction                               1   
 4              Consultant                               9   
 
    nombre_heures_travailless  annee_experience_totale  \
 0                         80                        8   
 1                         80                       10   
 2                         80                        7   
 3                 

## Transformations des variables

Nous appliquons ici :
- mapping oui/non → 0/1 (cible + heures sup)
- conversion pourcentage → ratio
- création de nouvelles features (changement de poste, probabilités normalisées, évolution de note)
- suppression de colonnes constantes (non informatives)


In [2]:
df_feat = add_engineered_features(df)
df_feat.shape, df_feat.head()

((1470, 38),
    age  genre  revenu_mensuel statut_marital departement  \
 0   41      0            5993    Célibataire  Commercial   
 1   49      1            5130       Marié(e)  Consulting   
 2   37      1            2090    Célibataire  Consulting   
 3   33      0            2909       Marié(e)  Consulting   
 4   27      1            3468       Marié(e)  Consulting   
 
                     poste  nombre_experiences_precedentes  \
 0        Cadre Commercial                               8   
 1  Assistant de Direction                               1   
 2              Consultant                               6   
 3  Assistant de Direction                               1   
 4              Consultant                               9   
 
    annee_experience_totale  annees_dans_l_entreprise  \
 0                        8                         6   
 1                       10                        10   
 2                        7                         0   
 3               

## Contrôles de cohérence

Nous vérifions des contraintes logiques (proxy de qualité) :
- annee_experience_totale >= annees_dans_l_entreprise >= annees_dans_le_poste_actuel
- cas particulier : si 0 expérience précédente, exp_totale - ancienneté devrait être faible (proxy d’anomalie)

Ces contrôles ne servent pas forcément à supprimer des lignes, mais à documenter la qualité du dataset.


In [3]:
metrics = compute_incoherence_metrics(df_feat)
metrics

{'incoherence_hierarchy_count': 0.0,
 'incoherence_hierarchy_ratio': 0.0,
 'gap_exp_minus_tenure_when_no_prev_ratio': 1.0}

## Comparaisons statistiques (attrition vs non-attrition)

Pour chaque variable numérique disponible :
- si distributions compatibles avec une hypothèse gaussienne → t-test de Welch
- sinon → Mann–Whitney

Nous appliquons ensuite une correction FDR (Benjamini–Hochberg) pour limiter les faux positifs liés aux tests multiples.


In [4]:
TARGET = "a_quitte_l_entreprise"

# sélection automatique des colonnes numériques (hors cible)
numeric_cols = [
    c for c in df_feat.columns if c != TARGET and pd.api.types.is_numeric_dtype(df_feat[c])
]

stats_table = compare_groups_univariate(df_feat, target=TARGET, cols=numeric_cols)
stats_table.head(20)

g:\Mon Drive\OC\Projet_4\technova_attrition\.venv\Lib\site-packages\scipy\stats\_stats_py.py:1687: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  b2 = skew(a, axis, _no_deco=True)
g:\Mon Drive\OC\Projet_4\technova_attrition\.venv\Lib\site-packages\scipy\stats\_stats_py.py:1792: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  b2 = kurtosis(a, axis, fisher=False, _no_deco=True)


,feature,test,statistic,p_value,mean_group0,mean_group1,mean_diff,effect_size_cohens_d_if_t,p_value_fdr
14,heure_supplementaires,mann_whitney,102061.5,3.985654e-21,0.234388,0.535865,0.301477,NaN,1.155840e-19
4,annee_experience_totale,mann_whitney,191654.0,2.399569e-14,11.862936,8.244726,-3.618210,NaN,2.852470e-13
2,revenu_mensuel,mann_whitney,191600.5,2.950831e-14,6832.739659,4787.092827,-2045.646832,NaN,2.852470e-13
25,proba_chgt_experience_par_an,mann_whitney,98701.5,1.109940e-13,0.297303,0.537972,0.240670,NaN,8.047062e-13
5,annees_dans_l_entreprise,mann_whitney,189639.0,2.916191e-13,7.369019,5.130802,-2.238217,NaN,1.429211e-12
9,niveau_hierarchique_poste,mann_whitney,187491.5,2.956987e-13,2.145985,1.637131,-0.508855,NaN,1.429211e-12
6,annees_dans_le_poste_actuel,mann_whitney,187007.0,4.429560e-12,4.484185,2.902954,-1.581231,NaN,1.835103e-11
23,annes_sous_responsable_actuel,mann_whitney,185859.5,1.806754e-11,4.367397,2.852321,-1.515076,NaN,6.549484e-11
16,nombre_participation_pee,mann_whitney,182610.0,4.013375e-11,0.845093,0.527426,-0.317667,NaN,1.293199e-10
0,age,mann_whitney,185362.0,5.304342e-11,37.561233,33.607595,-3.953638,NaN,1.538259e-10


In [5]:
# Sauvegarde tableau stats
stats_path = PATHS.reports / "univariate_tests.csv"
stats_table.to_csv(stats_path, index=False)

# Sauvegarde dataset final pour la modélisation
out_path = PATHS.data_processed / "employees_features.parquet"
df_feat.to_parquet(out_path, index=False)

stats_path, out_path

(WindowsPath('G:/Mon Drive/OC/Projet_4/technova_attrition/reports/univariate_tests.csv'),
 WindowsPath('G:/Mon Drive/OC/Projet_4/technova_attrition/data/processed/employees_features.parquet'))

## Conclusion

Nous disposons maintenant d’un dataset enrichi, documenté et cohérent, prêt pour :
- la définition des familles de variables (numériques, nominales, ordinales…)
- la construction d’un pipeline scikit-learn (ColumnTransformer)
- l’entraînement de plusieurs modèles (Dummy, linéaire, non-linéaire)
en tenant compte du déséquilibre de classes.
